In [1]:
import os, sys, subprocess, textwrap

PROJECT_DIR = os.path.expanduser("~/spotify_product_analytics_sql")
os.makedirs(PROJECT_DIR, exist_ok=True)
os.chdir(PROJECT_DIR)
print("Working directory:", os.getcwd())

# Install requirements (safe to re-run)
subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "pip"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "pandas", "pyarrow", "datasets", "tqdm"])


Working directory: /Users/Phillip/spotify_product_analytics_sql
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 3.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 25.1.1
    Uninstalling pip-25.1.1:
      Successfully uninstalled pip-25.1.1


0

In [2]:
import os
from datasets import load_dataset
from tqdm.auto import tqdm

OUT_DIR = "data"
os.makedirs(OUT_DIR, exist_ok=True)

DATASET_ID = "matthewfranglen/lastfm-1k"

ds = load_dataset(DATASET_ID)
print("Available splits:", list(ds.keys()))  # should be ['train','valid','test']

def export_split(split_name: str, out_csv: str, chunksize: int = 500_000):
    d = ds[split_name]
    cols = list(d.column_names)

    # write header
    with open(out_csv, "w", encoding="utf-8") as f:
        f.write(",".join(cols) + "\n")

    total = len(d)
    for start in tqdm(range(0, total, chunksize), desc=f"Export {split_name}"):
        end = min(start + chunksize, total)
        batch = d.select(range(start, end)).to_pandas()
        batch.to_csv(out_csv, mode="a", index=False, header=False)

for split in ds.keys():
    export_split(split, os.path.join(OUT_DIR, f"events_{split}.csv"))

print("Wrote:", sorted([f for f in os.listdir(OUT_DIR) if f.startswith("events_")]))


/Users/Phillip/.pyenv/versions/3.11.8/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Available splits: ['train', 'valid', 'test']


Export test: 100%|██████████| 5/5 [00:10<00:00,  2.20s/it]

Wrote: ['events_test.csv', 'events_train.csv', 'events_valid.csv']


In [3]:
import os
import pandas as pd
from tqdm.auto import tqdm

IN_DIR = "data"
OUT_DIR = "data"
os.makedirs(OUT_DIR, exist_ok=True)

# detect which exported files exist
SPLIT_FILES = []
for split in ("train", "valid", "test"):
    path = os.path.join(IN_DIR, f"events_{split}.csv")
    if os.path.exists(path):
        SPLIT_FILES.append((split, path))

if not SPLIT_FILES:
    raise FileNotFoundError("No exported events CSVs found. Run Block 1 first.")

USER_COL_CANDIDATES = ["user_id", "user"]
TS_COL_CANDIDATES = ["timestamp", "played_at", "time", "datetime", "date_time", "utc_time"]

ARTIST_ID_CANDIDATES = ["artist_id", "musicbrainz_artist_id"]
ARTIST_NAME_CANDIDATES = ["artist_name", "artist"]

TRACK_ID_CANDIDATES = ["track_id", "musicbrainz_track_id"]
TRACK_NAME_CANDIDATES = ["track_name", "track", "title"]

PROFILE_COUNTRY_CANDIDATES = ["country"]
PROFILE_SIGNUP_CANDIDATES = ["signup", "signup_date", "registration_date"]
PROFILE_GENDER_CANDIDATES = ["gender"]
PROFILE_AGE_CANDIDATES = ["age"]

def pick_col(cols, candidates):
    for c in candidates:
        if c in cols:
            return c
    return None

def load_in_chunks(path, chunksize=300_000):
    return pd.read_csv(path, chunksize=chunksize)

plays_frames = []
users_frames = []

for split_name, path in SPLIT_FILES:
    for chunk in tqdm(load_in_chunks(path), desc=f"Read {split_name}", unit="chunk"):
        cols = list(chunk.columns)

        ucol = pick_col(cols, USER_COL_CANDIDATES)
        tcol = pick_col(cols, TS_COL_CANDIDATES)
        if ucol is None or tcol is None:
            raise ValueError(
                f"Could not find user/timestamp columns in {path}. "
                f"First columns: {cols[:30]}"
            )

        aid = pick_col(cols, ARTIST_ID_CANDIDATES)
        anm = pick_col(cols, ARTIST_NAME_CANDIDATES)
        tid = pick_col(cols, TRACK_ID_CANDIDATES)
        tnm = pick_col(cols, TRACK_NAME_CANDIDATES)

        country = pick_col(cols, PROFILE_COUNTRY_CANDIDATES)
        signup  = pick_col(cols, PROFILE_SIGNUP_CANDIDATES)
        gender  = pick_col(cols, PROFILE_GENDER_CANDIDATES)
        age     = pick_col(cols, PROFILE_AGE_CANDIDATES)

        plays = pd.DataFrame({
            "user_id": chunk[ucol].astype(str),
            "played_at": pd.to_datetime(chunk[tcol], errors="coerce", utc=True),
            "artist_id": chunk[aid].astype(str) if aid else None,
            "artist_name": chunk[anm].astype(str) if anm else None,
            "track_id": chunk[tid].astype(str) if tid else None,
            "track_name": chunk[tnm].astype(str) if tnm else None,
            "split": split_name,
        }).dropna(subset=["user_id", "played_at"])

        plays_frames.append(plays)

        # users (best-effort if metadata exists)
        u = pd.DataFrame({"user_id": chunk[ucol].astype(str)})
        u["country"] = chunk[country].astype(str) if country else None
        u["signup_date"] = pd.to_datetime(chunk[signup], errors="coerce").dt.date if signup else pd.NaT
        u["gender"] = chunk[gender].astype(str) if gender else None
        u["age"] = pd.to_numeric(chunk[age], errors="coerce") if age else None
        users_frames.append(u)

plays_all = pd.concat(plays_frames, ignore_index=True).sort_values(["user_id", "played_at"])
plays_all.to_csv(os.path.join(OUT_DIR, "plays.csv"), index=False)

users_all = pd.concat(users_frames, ignore_index=True).drop_duplicates(subset=["user_id"])
users_all["plan"] = "free"  # synthetic; just for segmentation structure
users_all.to_csv(os.path.join(OUT_DIR, "users.csv"), index=False)

print("Wrote data/plays.csv and data/users.csv")
print("Rows - plays:", len(plays_all), "users:", len(users_all))


Read train: 35chunk [00:24,  1.47chunk/s]/Users/Phillip/.pyenv/versions/3.11.8/lib/python3.11/site-packages/tqdm/std.py:1181: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  for obj in iterable:
Read train: 45chunk [00:30,  1.47chunk/s]
Read valid: 6chunk [00:03,  1.71chunk/s]
Read test: 7chunk [00:04,  1.46chunk/s]


Wrote data/plays.csv and data/users.csv
Rows - plays: 16936134 users: 992


In [4]:
import pandas as pd

plays_preview = pd.read_csv("data/plays.csv", nrows=5)
users_preview = pd.read_csv("data/users.csv", nrows=5)

plays_preview, users_preview


(       user_id                  played_at  \
 0  user_000001  2006-08-13 13:59:20+00:00   
 1  user_000001  2006-08-13 14:03:29+00:00   
 2  user_000001  2006-08-13 14:10:43+00:00   
 3  user_000001  2006-08-13 14:17:40+00:00   
 4  user_000001  2006-08-13 14:19:06+00:00   
 
                               artist_id        artist_name  \
 0  09a114d9-7723-4e14-b524-379697f6d2b5  Plaid & Bob Jaroc   
 1  09a114d9-7723-4e14-b524-379697f6d2b5  Plaid & Bob Jaroc   
 2  09a114d9-7723-4e14-b524-379697f6d2b5  Plaid & Bob Jaroc   
 3  67fb65b5-6589-47f0-9371-8a40eb268dfb     Tommy Guerrero   
 4  1cfbc7d1-299c-46e6-ba4c-1facb84ba435      Artful Dodger   
 
                                track_id  \
 0  c4633ab1-e715-477f-8685-afa5f2058e42   
 1  bc2765af-208c-44c5-b3b0-cf597a646660   
 2  aa9c5a80-5cbe-42aa-a966-eb3cfa37d832   
 3  d9b1c1da-7e47-4f97-a135-77260f2f559d   
 4  120bb01c-03e4-465f-94a0-dce5e9fac711   
 
                                  track_name  split  
 0                 The